In [28]:
import pandas as pd

BP_train = pd.read_pickle("dataset/BP_train_solution.pkl")
BP_test = pd.read_pickle("dataset/BP_test_solution.pkl")


In [29]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import cross_val_score

X_train = BP_train.drop(columns=['score'])
Y_train = BP_train['score']

X_test = BP_test.drop(columns=['score'])
Y_test = BP_test['score']

rg = ExtraTreesRegressor(n_estimators=20, min_samples_leaf=20)
rg.fit(X_train, Y_train)

y_pred = rg.predict(X_test)
df = pd.DataFrame({'y_test': Y_test, 'y_pred': y_pred})
print(df.head())

scores = cross_val_score(rg, X_train, Y_train, cv=5, scoring='r2')
print("R^2 scores:", scores)

     y_test    y_pred
0 -1.657023 -1.885633
1 -1.657023 -1.885633
2 -1.657023 -1.885633
3 -1.657023 -1.885633
4 -1.657023 -1.885633
R^2 scores: [0.99998799 0.99999413 0.99996076 0.99999058 0.99997049]
